In [ ]:
import os
import librosa
import numpy
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

directory = os.listdir("/Users/nicolahaller/Documents/WI Master/Semester 2/Technology Lab/Samples/")
array = []

sequence_length = []
for sample_name in directory:
    if 'wav' in sample_name:
        file_path = os.path.join("/Users/nicolahaller/Documents/WI Master/Semester 2/Technology Lab/Samples/", sample_name)
        song, sr = librosa.load(file_path)
        sequence_length.append(len(song))
        array.append(song)


# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]


#Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)


batch_size = len(input_tensor)
timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))

# Test mit Conv1D Layer, MaxPooling1D Layer, Aktivierungsfunktion Sigmoid
# Mögliche Anpassungen: Anzahl der Samples, Aktivierungsfunktion 

# define model
model = Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3,strides=1, padding="causal", activation="sigmoid",input_shape=[None, 1]))
model.add(layers.MaxPooling1D(pool_size=2, strides=1, padding='same'))
model.add(LSTM(100, activation='sigmoid', return_sequences=True, input_shape=(timestep, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

# fit model
model.fit(input_data, output_data, epochs=25, verbose=2)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, None, 32)          128       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 32)         0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, None, 100)         53200     
                                                                 
 dense_1 (Dense)             (None, None, 1)           101       
                                                                 
Total params: 53,429
Trainable params: 53,429
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
1/1 - 32s - loss: 0.0202 - 32s/epoch - 32s/step
Epoch 2/25
1/1 - 28s - loss: 0.0320 - 28s/epoch - 28s/ste

In [ ]:
# demonstrate prediction
x_input = array[2]
x_input = x_input.reshape((1, timestep, 1))
yhat = model.predict(x_input, verbose=2)
print(yhat)


In [ ]:
print(array[3])


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(input_tensor, output_tensor, batch_size)
print("test loss, test acc:", results)